# Time series to supervised
Before machine learning can be used, time series forecasting problems must be re-framed as supervised learning problems. From a sequence to pairs of input and output sequences as described at https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [2]:
import boto3
import io
from sagemaker import get_execution_role

role = get_execution_role()
bucket ='sagemaker-data-energy-demand'

In [3]:
import numpy as np 
import pandas as pd 
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [4]:
def series_to_supervised(data,  col_names, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
     data: Sequence of observations as a list or NumPy array.
     n_in: Number of lag observations as input (X).
     n_out: Number of observations as output (y).
     dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
     Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (col_names[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    if i == 0:
        names += [('%s(t)' % (col_names[j])) for j in range(n_vars)]
    else:
        names += [('%s(t+%d)' % (col_names[j], i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [5]:
CITY = 'LosAngeles'
df_file = 'dataframes/%s_selectedfeatures.csv' % CITY

df_location = 's3://{}/{}'.format(bucket, df_file)
df = pd.read_csv(df_location, index_col ='datetime')
df.tail()

,dailycoolingdegreedays,dailyheatingdegreedays,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlyvisibility,hourlywindspeed,hourlycoolingdegrees,hourlyheatingdegrees,...,quarter,month,year,dayofyear,dayofmonth,weekofyear,isbusinessday,isholiday,daylight,demand
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-14 19:00:00+00:00,0.0,8.0,47.0,56.0,0.0,72.0,10.0,0.0,0.0,9.0,...,1,1,2020,14,14,3,1,0,0,3243.0
2020-01-14 20:00:00+00:00,0.0,8.0,47.0,55.0,0.0,74.0,10.0,0.0,0.0,10.0,...,1,1,2020,14,14,3,1,0,0,3207.0
2020-01-14 21:00:00+00:00,0.0,8.0,47.0,54.0,0.0,77.0,10.0,0.0,0.0,11.0,...,1,1,2020,14,14,3,1,0,0,3066.0
2020-01-14 22:00:00+00:00,0.0,8.0,46.0,53.0,0.0,77.0,10.0,0.0,0.0,12.0,...,1,1,2020,14,14,3,1,0,0,2978.0
2020-01-14 23:00:00+00:00,0.0,8.0,46.0,66.0,0.0,77.0,10.0,0.0,0.0,0.0,...,1,1,2020,14,14,3,1,0,0,2989.0


In [6]:
#set the column we want to predict (demand) to the first columns for consistency
values = df.values
# ensure all data is float
values = values.astype('float32')
# frame as supervised learning
reframed = series_to_supervised(values, list(df.columns), 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27]], axis=1, inplace=True)

cols = list(reframed.columns)
cols.remove('demand(t)')
cols.insert(0,'demand(t)')
reframed = reframed[cols]
reframed.index = df.index[:-1]


In [7]:
reframed.shape

(39807, 37)

In [8]:
reframed.tail()

,demand(t),dailycoolingdegreedays(t-1),dailyheatingdegreedays(t-1),hourlydewpointtemperature(t-1),hourlydrybulbtemperature(t-1),hourlyprecipitation(t-1),hourlyrelativehumidity(t-1),hourlyvisibility(t-1),hourlywindspeed(t-1),hourlycoolingdegrees(t-1),...,dayofweek(t),quarter(t),month(t),year(t),dayofyear(t),dayofmonth(t),weekofyear(t),isbusinessday(t),isholiday(t),daylight(t)
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-14 18:00:00+00:00,3243.0,0.0,8.0,47.0,57.0,0.0,69.0,10.0,0.0,0.0,...,1.0,1.0,1.0,2020.0,14.0,14.0,3.0,1.0,0.0,0.0
2020-01-14 19:00:00+00:00,3207.0,0.0,8.0,47.0,56.0,0.0,72.0,10.0,0.0,0.0,...,1.0,1.0,1.0,2020.0,14.0,14.0,3.0,1.0,0.0,0.0
2020-01-14 20:00:00+00:00,3066.0,0.0,8.0,47.0,55.0,0.0,74.0,10.0,0.0,0.0,...,1.0,1.0,1.0,2020.0,14.0,14.0,3.0,1.0,0.0,0.0
2020-01-14 21:00:00+00:00,2978.0,0.0,8.0,47.0,54.0,0.0,77.0,10.0,0.0,0.0,...,1.0,1.0,1.0,2020.0,14.0,14.0,3.0,1.0,0.0,0.0
2020-01-14 22:00:00+00:00,2989.0,0.0,8.0,46.0,53.0,0.0,77.0,10.0,0.0,0.0,...,1.0,1.0,1.0,2020.0,14.0,14.0,3.0,1.0,0.0,0.0


In [9]:
# save as csv file to continue in another notebook
s3_resource = boto3.resource('s3')

reframed_buffer = io.StringIO()
reframed_key = 'dataframes/%s_reframed.csv' % CITY
reframed.to_csv(reframed_buffer, compression=None)

s3_resource.Object(bucket, reframed_key).put(Body=reframed_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'B7F639732523D80C',
  'HostId': '/Y92bmAnVwHl+nlOyEv46oONTX8J40u/83Je595URFOoHG4wla7REr8UGiq66eQ8jl4hqlH5Vls=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/Y92bmAnVwHl+nlOyEv46oONTX8J40u/83Je595URFOoHG4wla7REr8UGiq66eQ8jl4hqlH5Vls=',
   'x-amz-request-id': 'B7F639732523D80C',
   'date': 'Fri, 17 Jan 2020 18:03:17 GMT',
   'etag': '"445ab0dd985a504b466d0b9ddda7f2ba"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"445ab0dd985a504b466d0b9ddda7f2ba"'}